In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import re
import random
import scipy
import re

from pathlib import Path
from pprint import pprint
from nltk.corpus import stopwords
from tqdm import tqdm

In [2]:
path_datasets = r"C:\Users\Alvaro Millan Ruiz\OneDrive\Escritorio\BDS\Block_5\NLP\Project"

In [3]:
import openpyxl  # neccesary to import the file CLASS

# Load the World Bank income index

In [4]:
# Load the csv files
wb_labels_until_2023 = pd.read_csv(r"C:\Users\Alvaro Millan Ruiz\OneDrive\Escritorio\BDS\Block_5\NLP\Project\NLP_BDS\WBIG.csv")

wb_labels_2024 = pd.read_excel("C:/Users/Alvaro Millan Ruiz/OneDrive/Escritorio/BDS/Block_5/NLP/Project/NLP_BDS/CLASS.xlsx")


Check out the labels of the income in both datasets

In [5]:
income_labels_until_2023 = wb_labels_until_2023['World Bank\'s income classification'].unique()
income_labels_until_2023

array(['Low-income countries', 'Lower-middle-income countries',
       'Upper-middle-income countries', 'High-income countries'],
      dtype=object)

In [6]:
income_labels_2024 = wb_labels_2024['Income group'].unique()
income_labels_2024

array(['Low income', 'Upper middle income', 'High income',
       'Lower middle income', nan], dtype=object)

Map the income levels into a categorical column for both datasets. 

1: low income ... 4: high income

In [7]:
# Map the income levels into a new column for both dataset
income_lable_map_2023 ={
    "Low-income countries": 1,
    "Lower-middle-income countries": 2,
    "Upper-middle-income countries": 3,
    "High-income countries": 4
}

income_lablel_map_2024 = {
    "Low income": 1,
    "Lower middle income": 2,
    "Upper middle income": 3,
    "High income": 4
}

wb_labels_until_2023['Income Level'] = wb_labels_until_2023['World Bank\'s income classification'].map(income_lable_map_2023).fillna(np.nan)
wb_labels_2024['Income Level'] = wb_labels_2024['Income group'].map(income_lablel_map_2024).astype('Int64')  # fill NaN if there is nothing

## Compare the ISO codes and which countries don't match in the datasets

In [8]:
iso_until_2023 = wb_labels_until_2023['Code'].unique()
len(iso_until_2023)

226

In [9]:
iso_2024 = wb_labels_2024['Code'].unique()
len(iso_2024)

218

In [10]:
not_in_until_2023 = [item for item in iso_2024 if item not in iso_until_2023]
print("Country codes in 2024, but not until 2023:", not_in_until_2023)

not_in_2024 = [item for item in iso_until_2023 if item not in iso_2024]
print("Country codes in until 2023, but not 2024:", not_in_2024)


Country codes in 2024, but not until 2023: ['CHI', 'XKX']
Country codes in until 2023, but not 2024: ['OWID_CIS', 'OWID_CZS', 'GUF', 'ATF', 'OWID_KOS', 'MYT', 'ANT', 'OWID_SRM', 'OWID_USS', 'OWID_YGS']


**Notes in this results** 

ANT -> Netherlands Antilles (stop existing untill 2009)

ATF -> French Southern Territories (Nobody cares)

GUF -> French Guaiana (Nobody cares) 

MYT -> Mayote (French Island nobody cares)

OWID_CIS -> Islands in the Channel of la Mancha (belongs to England)

OWID_CZS -> Czechoslovakia (from 1990 - 1991) Federative republic democratic anomaly 

OWID_SRM -> Serbia and Montenegro Wikipedia says: "The State Union of Serbia and Montenegro[a] or simply Serbia and Montenegro,[b] known until 2003 as the Federal Republic of Yugoslavia[c] and commonly referred to as FR Yugoslavia (FRY) or simply Yugoslavia,[d] was a country in Southeast Europe located in the Balkans that existed from 1992 to 2006" # IN THE UN DATA THIS IS REFERRED AS YUG

OWID_USS -> Czechoslovakia (from 1990 - 1991) Federative republic democratic anomaly 

OWID_YGS -> Yugoslavia disolved in 1992

OWID_KOS -> Kosovo, Exists Till this day


##############################################

CHI -> Channel Islands SAME AS OWID_CIS

XKX -> Kosovo, OWID_KOS

# Drop those countries that disolved

In [11]:
wb_labels_until_2023

,Entity,Code,Year,World Bank's income classification,Income Level
0,Afghanistan,AFG,1987,Low-income countries,1
1,Afghanistan,AFG,1988,Low-income countries,1
2,Afghanistan,AFG,1989,Low-income countries,1
3,Afghanistan,AFG,1990,Low-income countries,1
4,Afghanistan,AFG,1991,Low-income countries,1
...,...,...,...,...,...
7730,Zimbabwe,ZWE,2019,Lower-middle-income countries,2
7731,Zimbabwe,ZWE,2020,Lower-middle-income countries,2
7732,Zimbabwe,ZWE,2021,Lower-middle-income countries,2
7733,Zimbabwe,ZWE,2022,Lower-middle-income countries,2


In [12]:
drop_countries = ['OWID_CZS', 'GUF', 'MYT', 'ATF', 'MYT', 'ANT', 'OWID_USS', 'OWID_YGS']

wb_labels_until_2023_cleaned = wb_labels_until_2023[~wb_labels_until_2023['Code'].isin(drop_countries)]

# replace code OWID_CIS in Code for CHI
# Replace 'OWID_CIS' with 'CHI' in a specific column (e.g., 'Code')
wb_labels_until_2023_cleaned['Code'] = wb_labels_until_2023_cleaned['Code'].replace('OWID_CIS', 'CHI')
wb_labels_until_2023_cleaned['Code'] = wb_labels_until_2023_cleaned['Code'].replace('OWID_KOS', 'XKX')
wb_labels_until_2023_cleaned['Code'] = wb_labels_until_2023_cleaned['Code'].replace('OWID_SRM', 'YGS') # CHANGE THIS FOR THE YUG IN THE UN SPEECHES
wb_labels_until_2023_cleaned

C:\Users\Alvaro Millan Ruiz\AppData\Local\Temp\ipykernel_4736\1402715829.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_labels_until_2023_cleaned['Code'] = wb_labels_until_2023_cleaned['Code'].replace('OWID_CIS', 'CHI')
C:\Users\Alvaro Millan Ruiz\AppData\Local\Temp\ipykernel_4736\1402715829.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_labels_until_2023_cleaned['Code'] = wb_labels_until_2023_cleaned['Code'].replace('OWID_KOS', 'XKX')
C:\Users\Alvaro Millan Ruiz\AppData\Local\Temp\ipykernel

,Entity,Code,Year,World Bank's income classification,Income Level
0,Afghanistan,AFG,1987,Low-income countries,1
1,Afghanistan,AFG,1988,Low-income countries,1
2,Afghanistan,AFG,1989,Low-income countries,1
3,Afghanistan,AFG,1990,Low-income countries,1
4,Afghanistan,AFG,1991,Low-income countries,1
...,...,...,...,...,...
7730,Zimbabwe,ZWE,2019,Lower-middle-income countries,2
7731,Zimbabwe,ZWE,2020,Lower-middle-income countries,2
7732,Zimbabwe,ZWE,2021,Lower-middle-income countries,2
7733,Zimbabwe,ZWE,2022,Lower-middle-income countries,2


# Join the two World Bank datasets

We will take the country, code, year, and income level of both dataset and concatinate them into one global dataset

In [13]:
df_wb_until_2023 = wb_labels_until_2023_cleaned[['Entity', 'Code', 'Year', 'Income Level']] # take the useful columns in until_2023 dataset
df_wb_until_2023 = df_wb_until_2023.rename(columns={'Code': 'ISO-Code'})  # change the column name to ISO-Code so it matches the speeches dataset
df_wb_until_2023

,Entity,ISO-Code,Year,Income Level
0,Afghanistan,AFG,1987,1
1,Afghanistan,AFG,1988,1
2,Afghanistan,AFG,1989,1
3,Afghanistan,AFG,1990,1
4,Afghanistan,AFG,1991,1
...,...,...,...,...
7730,Zimbabwe,ZWE,2019,2
7731,Zimbabwe,ZWE,2020,2
7732,Zimbabwe,ZWE,2021,2
7733,Zimbabwe,ZWE,2022,2


In [16]:
df_wb_2024 = wb_labels_2024[['Economy', 'Code', 'Income Level']] # take the useful columns in 2024 dataset
df_wb_2024 = df_wb_2024.rename(columns={'Economy': 'Entity', 'Code': 'ISO-Code'})  # change the column name to ISO-Code so it matches the speeches dataset
df_wb_2024['Year'] = 2024  # create the 'Year' columns and fill it with 2024
df_wb_2024 = df_wb_2024[['Entity', 'ISO-Code', 'Year', 'Income Level']]
df_wb_2024

,Entity,ISO-Code,Year,Income Level
0,Afghanistan,AFG,2024,1
1,Albania,ALB,2024,3
2,Algeria,DZA,2024,3
3,American Samoa,ASM,2024,4
4,Andorra,AND,2024,4
...,...,...,...,...
213,Virgin Islands (U.S.),VIR,2024,4
214,West Bank and Gaza,PSE,2024,2
215,"Yemen, Rep.",YEM,2024,1
216,Zambia,ZMB,2024,2


## Now concatanate them

In [17]:
wb_labels_final = pd.concat([df_wb_until_2023, df_wb_2024], ignore_index=True)  # concatenate the two datasets
wb_labels_final

# save the final dataset as csv
wb_labels_final.to_csv("WB_labels_final.csv", index = False) 
wb_labels_final

,Entity,ISO-Code,Year,Income Level
0,Afghanistan,AFG,1987,1
1,Afghanistan,AFG,1988,1
2,Afghanistan,AFG,1989,1
3,Afghanistan,AFG,1990,1
4,Afghanistan,AFG,1991,1
...,...,...,...,...
7822,Virgin Islands (U.S.),VIR,2024,4
7823,West Bank and Gaza,PSE,2024,2
7824,"Yemen, Rep.",YEM,2024,1
7825,Zambia,ZMB,2024,2


# Matching the WB label to the speeches dataframe

In [18]:
# filter the label's dataset to only include years from 2000 onwards
wb_labels_final_1995 = wb_labels_final[wb_labels_final['Year'] >= 1995].reset_index()
wb_labels_final_1995

,index,Entity,ISO-Code,Year,Income Level
0,8,Afghanistan,AFG,1995,1
1,9,Afghanistan,AFG,1996,1
2,10,Afghanistan,AFG,1997,1
3,11,Afghanistan,AFG,1998,1
4,12,Afghanistan,AFG,1999,1
...,...,...,...,...,...
6348,7822,Virgin Islands (U.S.),VIR,2024,4
6349,7823,West Bank and Gaza,PSE,2024,2
6350,7824,"Yemen, Rep.",YEM,2024,1
6351,7825,Zambia,ZMB,2024,2


In [22]:
# load the speeches df (no mention to president and no stopwords)
df_speeches = pd.read_csv(path_datasets + "\\UN_Speeches_clean_wo_mention_to_president_and_stopwords.csv")

df_speeches_1995 = df_speeches[df_speeches['Year'] >= 1995].reset_index(drop=True)
df_speeches_1995


,Session,Year,ISO-Code,Speech,number_sentences,number_tokens
0,50,1995,AFG,eve fiftieth anniversary united nations repres...,200,3053
1,50,1995,AGO,permit outset sir behalf government republic a...,105,1518
2,50,1995,ALB,president general assembly fiftieth session ta...,75,1224
3,50,1995,AND,right begin thanking outgoing president minist...,122,1757
4,50,1995,ARE,behalf delegation united arab emirates gives g...,52,1108
...,...,...,...,...,...,...
5679,79,2024,WSM,excellencies extend congratulations excellency...,68,815
5680,79,2024,YEM,ladies gentlemen happy coincidence address tod...,58,986
5681,79,2024,ZAF,president 79th session un general assembly mr....,99,929
5682,79,2024,ZMB,ladies gentlemen congratulate excellency assum...,78,1216


# Check if ISO code are the same for both DF

In [31]:
df_speeches_1995_code_list = df_speeches_1995['ISO-Code'].unique() 
wb_labels_final_1995_list = wb_labels_final_1995['ISO-Code'].unique()

not_in_speeches = [item for item in wb_labels_final_1995_list if item not in df_speeches_1995_code_list]
print("Country codes in WB, but not in UN speeches:", not_in_speeches)

not_in_wb = [item for item in df_speeches_1995_code_list if item not in wb_labels_final_1995_list]
print("Country codes in UN speeches, but not in WB:", not_in_wb)


Country codes in WB, but not in UN speeches: ['ASM', 'ABW', 'BMU', 'VGB', 'CYM', 'CHI', 'CUW', 'FRO', 'PYF', 'GIB', 'GRL', 'GUM', 'HKG', 'IMN', 'XKX', 'MAC', 'NCL', 'MNP', 'PRI', 'MAF', 'SXM', 'TWN', 'TCA', 'VIR']
Country codes in UN speeches, but not in WB: ['YUG', 'VAT', 'EU']


In [19]:
df_speeches_1995[df_speeches_1995['ISO-Code'] == 'YUG']

,index,Session,Year,ISO-Code,Speech,number_sentences,number_tokens
1254,6522,56,2001,YUG,president general assembly sterling credential...,62,725
1442,6710,57,2002,YUG,great honour privilege first democratically el...,56,715
1631,6899,58,2003,YUG,honoured address first time behalf state union...,43,403
1823,7091,59,2004,YUG,allow outset convey greetings best wishes peop...,32,432
2008,7276,60,2005,YUG,seven days ago rostrum warned situation kosovo...,32,396


# WATCH OUT WITH THE Non-Self-Governing Territory

In [20]:
def add_income_level(speeches_df, income_df):
    # merge on ISO-Code and Year
    merged = speeches_df.merge(
        income_df[['ISO-Code', 'Year', 'Income Level']],
        how='left',
        on=['ISO-Code', 'Year']
    )
    
    # fill missing income levels with 0
    merged['Income Level'] = merged['Income Level'].fillna(0).astype(int)
    
    return merged

df_combined = add_income_level(df_speeches_1995, wb_labels_final_1995)
df_combined

,index,Session,Year,ISO-Code,Speech,number_sentences,number_tokens,Income Level
0,5268,50,1995,AFG,eve fiftieth anniversary united nations repres...,200,3053,1
1,5269,50,1995,AGO,permit outset sir behalf government republic a...,105,1518,1
2,5270,50,1995,ALB,president general assembly fiftieth session ta...,75,1224,1
3,5271,50,1995,AND,right begin thanking outgoing president minist...,122,1757,4
4,5272,50,1995,ARE,behalf delegation united arab emirates gives g...,52,1108,4
...,...,...,...,...,...,...,...,...
5679,10947,79,2024,WSM,excellencies extend congratulations excellency...,68,815,2
5680,10948,79,2024,YEM,ladies gentlemen happy coincidence address tod...,58,986,1
5681,10949,79,2024,ZAF,president 79th session un general assembly mr....,99,929,3
5682,10950,79,2024,ZMB,ladies gentlemen congratulate excellency assum...,78,1216,2


In [21]:
len(df_combined[df_combined['Income Level'] == 0])

73

In [22]:
list_not_in_WB = df_combined[df_combined['Income Level'] == 0]['ISO-Code'].unique()
list_not_in_WB

array(['SMR', 'NRU', 'TUV', 'YUG', 'VAT', 'EU', 'VEN'], dtype=object)

In [26]:
df_combined[df_combined['ISO-Code'] == 'YUG']

,index,Session,Year,ISO-Code,Speech,number_sentences,number_tokens,Income Level
1254,6522,56,2001,YUG,president general assembly sterling credential...,62,725,0
1442,6710,57,2002,YUG,great honour privilege first democratically el...,56,715,0
1631,6899,58,2003,YUG,honoured address first time behalf state union...,43,403,0
1823,7091,59,2004,YUG,allow outset convey greetings best wishes peop...,32,432,0
2008,7276,60,2005,YUG,seven days ago rostrum warned situation kosovo...,32,396,0
